<a href="https://colab.research.google.com/github/ooutaksaluk/MBT-analysis/blob/main/Main_Battle_Tank_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**Main Battle Tank analysis**



In [1]:
!pip install anvil-uplink opencv-python-headless numpy torch torchvision
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu121/torch2.0/index.html

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu121/torch2.0/index.html
ERROR: Could not find a version that satisfies the requirement detectron2 (from versions: none)
ERROR: No matching distribution found for detectron2


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os

train_dir = '/content/drive/My Drive/dataset/train'
validation_dir = '/content/drive/My Drive/dataset/validation'

# ตรวจสอบว่าโฟลเดอร์มีอยู่จริง
if os.path.exists(train_dir):
    print("พบโฟลเดอร์:", train_dir)
else:
    print("ไม่พบโฟลเดอร์:", train_dir)


พบโฟลเดอร์: /content/drive/My Drive/dataset/train


**เตรียมข้อมูลและการโหลดข้อมูลด้วย Keras**

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # ใช้ 20% เป็น validation
)

# โหลดข้อมูล Train (ใช้ 80%)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

# โหลดข้อมูล Validation (ใช้ 20%)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)



Found 72 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


**Training Model**

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# สร้างโมเดล CNN
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='sigmoid')  # 2 Output: Sight, Mantlet
])

# ใช้ binary_crossentropy เพราะเป็น Multi-Label (sigmoid)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.save('/content/drive/My Drive/dataset/trained_model.h5')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
for layer in model.layers:
    print(layer.name)

conv2d
max_pooling2d
conv2d_1
max_pooling2d_1
conv2d_2
max_pooling2d_2
flatten
dense
dropout
dense_1


In [ ]:
import torch
import torchvision

# โหลดโมเดล Faster R-CNN ที่ถูก fine-tune แล้ว
num_classes = 3  # (background, sight, mantlet)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# โหลดน้ำหนักโมเดลที่ฝึกแล้ว
model.load_state_dict(torch.load("trained_fasterrcnn.pth", map_location=torch.device("cpu")))
model.eval()

In [ ]:
import cv2
import numpy as np
from torchvision.transforms import functional as F

def detect_objects(image_path, model, threshold=0.5):
    # โหลดภาพ
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # แปลงเป็น tensor
    img_tensor = F.to_tensor(img_rgb).unsqueeze(0)

    # ใช้โมเดลพยากรณ์
    with torch.no_grad():
        predictions = model(img_tensor)[0]

    # ดึงค่าพิกัด Bounding Boxes
    pred_boxes = predictions["boxes"].cpu().numpy()
    pred_classes = predictions["labels"].cpu().numpy()
    pred_scores = predictions["scores"].cpu().numpy()

    # แสดงผลลัพธ์
    for i in range(len(pred_scores)):
        if pred_scores[i] >= threshold:
            x_min, y_min, x_max, y_max = map(int, pred_boxes[i])
            class_id = pred_classes[i]
            confidence = pred_scores[i]

            # ใส่ Label ชื่อคลาส
            label = "sight" if class_id == 1 else "mantlet"
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(img, f"{label} ({confidence:.2f})",
                        (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)

    # แสดงผลลัพธ์
    cv2.imshow("Detected Objects", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [8]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image
import tensorflow as tf



# กำหนดชื่อเลเยอร์ Convolution สุดท้าย
LAST_CONV_LAYER = "conv2d_2"

def make_gradcam_heatmap(model, img_array, last_conv_layer_name):
    """ สร้าง Heatmap จาก Grad-CAM """
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    for i in range(conv_outputs.shape[-1]):
        conv_outputs[:, :, i] *= pooled_grads[i]

    heatmap = np.mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    return heatmap

def overlay_heatmap(img, heatmap, alpha=0.4):
    """ ซ้อน Heatmap ลงบนภาพ """
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    return superimposed_img




**Anvil**



In [10]:
import anvil.server
import anvil.media
import torch
import torchvision
from torchvision.transforms import functional as F
import numpy as np
import cv2

# เชื่อมต่อกับ Anvil
anvil.server.connect("server_LX2D76OV3RCR3XCSOMCFYQZL-JGQCERXRR5DWB5QH")

model = tf.keras.models.load_model('/content/drive/My Drive/dataset/trained_model.h5')

@anvil.server.callable
def process_image(file):
    print("เริ่มประมวลผลรูปภาพ...")

    # โหลดรูปภาพจาก Anvil
    file_bytes = file.get_bytes()
    nparr = np.frombuffer(file_bytes, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    if img is None:
        print("ไม่สามารถโหลดรูปภาพได้!")
        return None

    print("โหลดภาพสำเร็จ กำลังประมวลผล...")

    # แปลงภาพให้เข้ากับโมเดล Faster R-CNN
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # แปลงเป็น RGB
    img_tensor = F.to_tensor(img_rgb)  # แปลงเป็น tensor
    img_tensor = img_tensor.unsqueeze(0)  # เพิ่มมิติ batch

    # พยากรณ์ Bounding Box ด้วย Faster R-CNN
    with torch.no_grad():
        prediction = model(img_tensor)[0]

    pred_boxes = prediction["boxes"].cpu().numpy()
    pred_classes = prediction["labels"].cpu().numpy()
    pred_scores = prediction["scores"].cpu().numpy()

    # กำหนด threshold ความมั่นใจของผลลัพธ์ (เช่น 0.5)
    threshold = 0.5
    for i in range(len(pred_scores)):
        if pred_scores[i] >= threshold:
            x_min, y_min, x_max, y_max = map(int, pred_boxes[i])
            class_id = int(pred_classes[i])
            confidence = float(pred_scores[i])

            # วาดกรอบ Bounding Box
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(img, f"Class {class_id} ({confidence:.2f})",
                        (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.5, (0, 255, 0), 2)

    #  ส่งผลลัพธ์กลับไปยัง Anvil
    _, img_encoded = cv2.imencode('.jpg', img)
    return anvil.BlobMedia("image/jpeg", img_encoded.tobytes())


#  เริ่มเซิร์ฟเวอร์ Anvil
anvil.server.wait_forever()

เริ่มประมวลผลรูปภาพ...
โหลดภาพสำเร็จ กำลังประมวลผล...


KeyboardInterrupt: 